In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,  plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  f1_score, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, RandomForestRegressor
from sklearn.compose import make_column_transformer, make_column_selector
import numpy as np
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv('train.csv')

In [3]:
X , y = df.drop(['User_ID', 'Product_ID', 'Purchase'], axis =1) , df['Purchase']
X.dtypes

Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
dtype: object

In [4]:
X['Product_Category_1'] = X['Product_Category_1'].astype('str')
X['Product_Category_2'] = X['Product_Category_2'].astype('str')
X['Product_Category_3'] = X['Product_Category_3'].astype('str')

In [5]:
si = SimpleImputer(strategy='constant', fill_value='Unknown').set_output(transform='pandas')
X[['Product_Category_2','Product_Category_3']] = si.fit_transform(X[['Product_Category_2','Product_Category_3']])

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.3, random_state=24)
X_train.dtypes

Gender                        object
Age                           object
Occupation                     int64
City_Category                 object
Stay_In_Current_City_Years    object
Marital_Status                 int64
Product_Category_1            object
Product_Category_2            object
Product_Category_3            object
dtype: object

In [7]:
X_ohe = OneHotEncoder(sparse_output=False,drop='first', handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer((X_ohe,make_column_selector(dtype_include=object)),
                            ('passthrough',make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X_train)
X_tst = ct.transform(X_test)

In [8]:
X_trn.shape

(385047, 66)

## With XGBoost

In [10]:
n_trees = [25,50,100]
l_rate = [0.1,0.4,0.6,1]
depth = [3,5,None]

scores = []
for i in tqdm(range(len(n_trees))):
    for r in l_rate:
        for d in depth:
            xgb = XGBRegressor(random_state=24,max_depth=d,learning_rate=r,n_estimators=n_trees[i])
            xgb.fit(X_trn,Y_train)
            y_pred = xgb.predict(X_tst)
            scores.append([n_trees[i],r,d,r2_score(Y_test,y_pred)])

df1 = pd.DataFrame(scores , columns = ['trees','l_rate','depth', 'r2_score'])
df1.sort_values('r2_score', ascending=False).iloc[0]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:37<00:00, 12.50s/it]


trees       100.000000
l_rate        0.600000
depth              NaN
r2_score      0.673323
Name: 32, dtype: float64

## With Light GBM

In [12]:
n_trees = [25,50,100]
l_rate = [0.1,0.4,0.6,1]
depth = [3,5,None]

scores = []
for i in tqdm(range(len(n_trees))):
    for r in l_rate:
        for d in depth:
            lgb = LGBMRegressor(random_state=24,max_depth=d,learning_rate=r,n_estimators=n_trees[i])
            lgb.fit(X_trn,Y_train)
            y_pred = lgb.predict(X_tst)
            scores.append([n_trees[i],r,d,r2_score(Y_test,y_pred)])

df1 = pd.DataFrame(scores , columns = ['trees','l_rate','depth', 'r2_score'])
df1.sort_values('r2_score', ascending=False).iloc[0]

  0%|                                                                                                                                                                                                                 | 0/3 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004961 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 151
[LightGBM] [Info] Number of data points in the train set: 385047, number of used features: 66
[LightGBM] [Info] Start training from score 9261.376115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

 33%|███████████████████████████████████████████████████████████████████                                                                                                                                      | 1/3 [00:06<00:12,  6.38s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004086 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 151
[LightGBM] [Info] Number of data points in the train set: 385047, number of used features: 66
[LightGBM] [Info] Start training from score 9261.376115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                   | 2/3 [00:13<00:07,  7.01s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003832 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 151
[LightGBM] [Info] Number of data points in the train set: 385047, number of used features: 66
[LightGBM] [Info] Start training from score 9261.376115
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:23<00:00,  7.95s/it]


trees       100.000000
l_rate        0.600000
depth              NaN
r2_score      0.675722
Name: 32, dtype: float64

## With CatBoost

In [14]:
n_trees = [25,50,100]
l_rate = [0.1,0.4,0.6,1]
depth = [3,5,None]

scores = []
for i in tqdm(range(len(n_trees))):
    for r in l_rate:
        for d in depth:
            cgb = CatBoostRegressor(random_state=24,max_depth=d,learning_rate=r,n_estimators=n_trees[i])
            cgb.fit(X_trn,Y_train)
            y_pred = cgb.predict(X_tst)
            scores.append([n_trees[i],r,d,r2_score(Y_test,y_pred)])

df1 = pd.DataFrame(scores , columns = ['trees','l_rate','depth', 'r2_score'])
df1.sort_values('r2_score', ascending=False).iloc[0]

  0%|                                                                                                                                                                                                                 | 0/3 [00:00<?, ?it/s]

0:	learn: 4864.4045738	total: 194ms	remaining: 4.65s
1:	learn: 4731.4841361	total: 228ms	remaining: 2.62s
2:	learn: 4614.3718417	total: 249ms	remaining: 1.83s
3:	learn: 4516.1368015	total: 266ms	remaining: 1.39s
4:	learn: 4429.3203739	total: 281ms	remaining: 1.12s
5:	learn: 4359.8351658	total: 297ms	remaining: 941ms
6:	learn: 4290.6810299	total: 312ms	remaining: 802ms
7:	learn: 4233.6297217	total: 328ms	remaining: 697ms
8:	learn: 4174.2575110	total: 342ms	remaining: 609ms
9:	learn: 4124.4024467	total: 357ms	remaining: 536ms
10:	learn: 4075.0929667	total: 372ms	remaining: 474ms
11:	learn: 4029.9256194	total: 387ms	remaining: 419ms
12:	learn: 3984.0154356	total: 404ms	remaining: 373ms
13:	learn: 3945.8167986	total: 419ms	remaining: 329ms
14:	learn: 3909.5866255	total: 436ms	remaining: 291ms
15:	learn: 3877.3459953	total: 451ms	remaining: 253ms
16:	learn: 3844.1379485	total: 465ms	remaining: 219ms
17:	learn: 3814.6277734	total: 480ms	remaining: 187ms
18:	learn: 3787.7226841	total: 494ms	r

 33%|███████████████████████████████████████████████████████████████████                                                                                                                                      | 1/3 [00:08<00:16,  8.17s/it]

22:	learn: 2922.6599520	total: 460ms	remaining: 40ms
23:	learn: 2920.9030211	total: 483ms	remaining: 20.1ms
24:	learn: 2919.4729226	total: 503ms	remaining: 0us
0:	learn: 4864.4045738	total: 16.3ms	remaining: 798ms
1:	learn: 4731.4841361	total: 32ms	remaining: 769ms
2:	learn: 4614.3718417	total: 48.6ms	remaining: 761ms
3:	learn: 4516.1368015	total: 64.8ms	remaining: 745ms
4:	learn: 4429.3203739	total: 80.3ms	remaining: 723ms
5:	learn: 4359.8351658	total: 96.1ms	remaining: 705ms
6:	learn: 4290.6810299	total: 112ms	remaining: 685ms
7:	learn: 4233.6297217	total: 127ms	remaining: 667ms
8:	learn: 4174.2575110	total: 143ms	remaining: 653ms
9:	learn: 4124.4024467	total: 158ms	remaining: 632ms
10:	learn: 4075.0929667	total: 173ms	remaining: 615ms
11:	learn: 4029.9256194	total: 189ms	remaining: 599ms
12:	learn: 3984.0154356	total: 205ms	remaining: 585ms
13:	learn: 3945.8167986	total: 219ms	remaining: 564ms
14:	learn: 3909.5866255	total: 236ms	remaining: 551ms
15:	learn: 3877.3459953	total: 251ms

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                   | 2/3 [00:21<00:11, 11.31s/it]

0:	learn: 4864.4045738	total: 19.6ms	remaining: 1.94s
1:	learn: 4731.4841361	total: 39.4ms	remaining: 1.93s
2:	learn: 4614.3718417	total: 57.6ms	remaining: 1.86s
3:	learn: 4516.1368015	total: 76.2ms	remaining: 1.83s
4:	learn: 4429.3203739	total: 94.4ms	remaining: 1.79s
5:	learn: 4359.8351658	total: 113ms	remaining: 1.77s
6:	learn: 4290.6810299	total: 131ms	remaining: 1.74s
7:	learn: 4233.6297217	total: 149ms	remaining: 1.71s
8:	learn: 4174.2575110	total: 166ms	remaining: 1.68s
9:	learn: 4124.4024467	total: 184ms	remaining: 1.65s
10:	learn: 4075.0929667	total: 201ms	remaining: 1.63s
11:	learn: 4029.9256194	total: 219ms	remaining: 1.6s
12:	learn: 3984.0154356	total: 238ms	remaining: 1.59s
13:	learn: 3945.8167986	total: 255ms	remaining: 1.56s
14:	learn: 3909.5866255	total: 272ms	remaining: 1.54s
15:	learn: 3877.3459953	total: 288ms	remaining: 1.51s
16:	learn: 3844.1379485	total: 305ms	remaining: 1.49s
17:	learn: 3814.6277734	total: 321ms	remaining: 1.46s
18:	learn: 3787.7226841	total: 335

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:47<00:00, 17.97s/it]

93:	learn: 2862.8670956	total: 2.17s	remaining: 139ms
94:	learn: 2862.4674277	total: 2.2s	remaining: 116ms
95:	learn: 2862.0634918	total: 2.22s	remaining: 92.7ms
96:	learn: 2861.6407278	total: 2.25s	remaining: 69.6ms
97:	learn: 2861.1420968	total: 2.27s	remaining: 46.4ms
98:	learn: 2860.6922787	total: 2.29s	remaining: 23.2ms
99:	learn: 2860.4321234	total: 2.31s	remaining: 0us


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:47<00:00, 15.86s/it]


trees       100.000000
l_rate        1.000000
depth              NaN
r2_score      0.672455
Name: 35, dtype: float64

In [15]:
X_ohe = OneHotEncoder(sparse_output=False,drop='first', handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer((X_ohe,make_column_selector(dtype_include=object)),
                            ('passthrough',make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X_train)
X_tst = ct.transform(X_test)

# Stacking 

In [17]:


from sklearn.ensemble import StackingRegressor

lgbm = LGBMRegressor()
rf = RandomForestRegressor()
dtc = DecisionTreeRegressor()
cb = CatBoostRegressor()



In [18]:

stack = StackingRegressor(estimators=[('rf', rf) , ('dtc', dtc), ('Cb', cb)] , final_estimator=lgbm , passthrough=True)
stack.fit(X_trn , Y_train)
y_pred = stack.predict(X_tst)
print(r2_score(Y_test , y_pred))

Learning rate set to 0.10488
0:	learn: 4761.5497120	total: 22.1ms	remaining: 22.1s
1:	learn: 4541.9642367	total: 43.4ms	remaining: 21.7s
2:	learn: 4355.4050321	total: 64.9ms	remaining: 21.6s
3:	learn: 4199.9437569	total: 87ms	remaining: 21.7s
4:	learn: 4067.3160210	total: 109ms	remaining: 21.6s
5:	learn: 3955.9963283	total: 131ms	remaining: 21.7s
6:	learn: 3869.5301207	total: 153ms	remaining: 21.7s
7:	learn: 3783.5714213	total: 176ms	remaining: 21.8s
8:	learn: 3716.3434455	total: 202ms	remaining: 22.2s
9:	learn: 3659.8695172	total: 227ms	remaining: 22.4s
10:	learn: 3599.0710261	total: 249ms	remaining: 22.4s
11:	learn: 3553.5709707	total: 268ms	remaining: 22.1s
12:	learn: 3502.4855570	total: 291ms	remaining: 22.1s
13:	learn: 3465.1152191	total: 311ms	remaining: 21.9s
14:	learn: 3423.5690489	total: 337ms	remaining: 22.1s
15:	learn: 3388.5715722	total: 361ms	remaining: 22.2s
16:	learn: 3357.6824600	total: 382ms	remaining: 22.1s
17:	learn: 3333.1987999	total: 400ms	remaining: 21.8s
18:	lea

In [19]:
X_trn.shape

(385047, 66)

In [20]:
X_tst.shape

(165021, 66)

In [21]:
Y_train.shape

(385047,)

## Inferencing

### Light GBM gives the best results

In [24]:
test = pd.read_csv('test_Vges7qu.csv')

In [25]:
test.dtypes

Comb                           object
User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
dtype: object

In [26]:
test['Product_Category_1'] = test['Product_Category_1'].astype('str')
test['Product_Category_2'] = test['Product_Category_2'].astype('str')
test['Product_Category_3'] = test['Product_Category_3'].astype('str')

In [27]:
test.drop(['Comb','User_ID','Product_ID'], axis = 1)

Gender    Age  Occupation City_Category Stay_In_Current_City_Years  \
0           M  46-50           7             B                          2   
1           M  26-35          17             C                          0   
2           F  36-45           1             B                         4+   
3           F  36-45           1             B                         4+   
4           F  26-35           1             C                          1   
...       ...    ...         ...           ...                        ...   
233594      F  26-35          15             B                         4+   
233595      F  26-35          15             B                         4+   
233596      F  26-35          15             B                         4+   
233597      F  46-50           1             C                         4+   
233598      F  46-50           0             B                         4+   

        Marital_Status Product_Category_1 Product_Category_2  \
0                    0                  1               11.0   
1                    0                  3                5.0   
2                    0                  5               14.0   
3                    0                  4                9.0   
4                    0                  4                5.0   
...                ...                ...                ...   
233594               0                  8                nan   
233595               0                  5                8.0   
233596               0                  1                5.0   
233597               0                 10               16.0   
233598               0                  4                5.0   

       Product_Category_3  
0                     nan  
1                     nan  
2                     nan  
3                     nan  
4                    12.0  
...                   ...  
233594                nan  
233595                nan  
233596               12.0  
233597                nan  
233598                nan  

[233599 rows x 9 columns]

In [28]:
si = SimpleImputer(strategy='constant', fill_value='Unknown').set_output(transform='pandas')
test[['Product_Category_2','Product_Category_3']] = si.fit_transform(test[['Product_Category_2','Product_Category_3']])

In [29]:
test

Comb  User_ID Product_ID Gender    Age  Occupation  \
0       1000004P00128942  1000004  P00128942      M  46-50           7   
1       1000009P00113442  1000009  P00113442      M  26-35          17   
2       1000010P00288442  1000010  P00288442      F  36-45           1   
3       1000010P00145342  1000010  P00145342      F  36-45           1   
4       1000011P00053842  1000011  P00053842      F  26-35           1   
...                  ...      ...        ...    ...    ...         ...   
233594  1006036P00118942  1006036  P00118942      F  26-35          15   
233595  1006036P00254642  1006036  P00254642      F  26-35          15   
233596  1006036P00031842  1006036  P00031842      F  26-35          15   
233597  1006037P00124742  1006037  P00124742      F  46-50           1   
233598  1006039P00316642  1006039  P00316642      F  46-50           0   

       City_Category Stay_In_Current_City_Years  Marital_Status  \
0                  B                          2               0   
1                  C                          0               0   
2                  B                         4+               0   
3                  B                         4+               0   
4                  C                          1               0   
...              ...                        ...             ...   
233594             B                         4+               0   
233595             B                         4+               0   
233596             B                         4+               0   
233597             C                         4+               0   
233598             B                         4+               0   

       Product_Category_1 Product_Category_2 Product_Category_3  
0                       1               11.0                nan  
1                       3                5.0                nan  
2                       5               14.0                nan  
3                       4                9.0                nan  
4                       4                5.0               12.0  
...                   ...                ...                ...  
233594                  8                nan                nan  
233595                  5                8.0                nan  
233596                  1                5.0               12.0  
233597                 10               16.0                nan  
233598                  4                5.0                nan  

[233599 rows x 12 columns]

In [30]:
X_ohe = OneHotEncoder(sparse_output=False,drop='first', handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer((X_ohe,make_column_selector(dtype_include=object)),
                            ('passthrough',make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False).set_output(transform='pandas')

X_trn = ct.fit_transform(X)

In [31]:
test1= ct.transform(test)

In [32]:
test1.shape

(233599, 66)

In [33]:
# Predicting
lgb = LGBMRegressor(random_state=24,max_depth=None,learning_rate=0.6,n_estimators=100)
lgb.fit(X_trn,y)
y_pred = lgb.predict(test1)

df1 = pd.DataFrame(y_pred,columns = ['Purchase'])
df1['User_ID'] = test['User_ID']
df1['Product_ID'] = test['Product_ID']

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005227 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 151
[LightGBM] [Info] Number of data points in the train set: 550068, number of used features: 66
[LightGBM] [Info] Start training from score 9263.968713


In [34]:
y_pred

array([15280.62943438, 10823.19802328,  7117.69482519, ...,
       13393.33202496, 19979.49424721,  2432.25514962])

In [35]:
submit = df1.to_csv('Submit.csv', index=False)